In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import sin, cos, pi, sqrt
import random
import time
import copy
import os
from rtsvg import *
rt = RACETrack()

In [ ]:
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'
os.path.exists(edges_filename), os.path.exists(layout_filename)

In [ ]:
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df  = pl.DataFrame(_lu_)
g   = rt.createNetworkXGraph(df, [('fm','to')])
pos = nx.spring_layout(g)
ln_params = {'relationships':[('fm','to')], 'pos':pos}
_rtg_ = rt.interactiveGraphLayout(df, ln_params, w=768, h=768)
if os.path.exists(layout_filename): _rtg_.loadLayout(layout_filename)
_rtg_

In [4]:
_rtg_.saveLayout(layout_filename)

In [ ]:
xmin,ymin,xmax,ymax = None, None, None, None
for _xy_ in pos.values():
    if xmin is None or _xy_[0] < xmin: xmin = _xy_[0]
    if ymin is None or _xy_[1] < ymin: ymin = _xy_[1]
    if xmax is None or _xy_[0] > xmax: xmax = _xy_[0]
    if ymax is None or _xy_[1] > ymax: ymax = _xy_[1]
xperc, yperc = (xmax-xmin)*0.05, (ymax-ymin)*0.05
xmin,ymin,xmax,ymax = xmin-xperc, ymin-yperc, xmax+xperc, ymax+yperc
print(xmin,ymin,xmax,ymax)

In [ ]:
def multiLevelEuclideanMST(pos, levels=3):
    _d_shortest_ = None
    _lu_ = {'fm':[], 'to':[]}
    edge_already_chosen = set()
    for _level_ in range(levels):
        g_this_round = nx.Graph()
        segment_already_seen_this_round = set()
        for k0 in pos.keys():
            xy0 = pos[k0]
            for k1 in pos.keys():
                if k0 == k1: continue
                xy1 = pos[k1]
                seg = (xy0,xy1)
                if seg in edge_already_chosen or \
                   seg in segment_already_seen_this_round: continue
                _d_ = sqrt((xy0[0] - xy1[0])**2 + (xy0[1] - xy1[1])**2)
                if _d_shortest_ is None or _d_ < _d_shortest_: _d_shortest_ = _d_
                g_this_round.add_edge(k0,k1,weight=_d_)
                segment_already_seen_this_round.add(seg)
                segment_already_seen_this_round.add((k1,k0))
        g_emst_this_round = nx.minimum_spanning_tree(g_this_round)
        for _edge_ in g_emst_this_round.edges: 
            _lu_['fm'].append(_edge_[0]), _lu_['to'].append(_edge_[1])
            edge_already_chosen.add((pos[_edge_[0]], pos[_edge_[1]]))
            edge_already_chosen.add((pos[_edge_[1]], pos[_edge_[0]]))
    return pl.DataFrame(_lu_), _d_shortest_

df_multi_emst, d_shortest = multiLevelEuclideanMST(pos, levels=2)

w, h = 600, 600
svg = [f'<svg width="{w}" height="{h}" x="0" y="0">']
xT,yT = lambda x: w*(x-xmin)/(xmax-xmin), lambda y: h - h*(y-ymin)/(ymax-ymin)
for key in pos.keys(): svg.append(f'<circle cx="{xT(pos[key][0])}" cy="{yT(pos[key][1])}" r="1.2" fill="#a0a0a0"/>')
for i in range(len(df_multi_emst)):
    _fm_,  _to_  = df_multi_emst['fm'][i], df_multi_emst['to'][i]
    _xy0_, _xy1_ = pos[_fm_], pos[_to_]
    svg.append(f'<line x1="{xT(_xy0_[0])}" y1="{yT(_xy0_[1])}" x2="{xT(_xy1_[0])}" y2="{yT(_xy1_[1])}" stroke-width="0.2" stroke="#a0a0a0" />')

#raster = [[None for x in range(w)] for y in range(h)]
#for key in pos.keys(): 
#    _x_, _y_ = int(xT(pos[key][0])), int(yT(pos[key][1]))
#    raster[_y_][_x_] = key
#raster_state, raster_found_time, raster_origin = rt.levelSetFast(raster)
#rt.svgObject(rt.levelSetStateAndFoundTimeSVG(raster_state, raster_found_time))

pts = []
for i in range(len(df_multi_emst)):
    _fm_,  _to_  = df_multi_emst['fm'][i], df_multi_emst['to'][i]
    _xy0_, _xy1_ = pos[_fm_], pos[_to_]
    _x_,   _y_   = (_xy0_[0] + _xy1_[0])/2.0, (_xy0_[1] + _xy1_[1])/2.0
    pts.append((_x_,_y_))
    svg.append(f'<circle cx="{xT(_x_)}" cy="{yT(_y_)}" r="1.2" fill="#ff0000"/>')
    _d_          = sqrt((_xy0_[0] - _xy1_[0])**2 + (_xy0_[1] - _xy1_[1])**2)
    if _d_ >= 4.0 * d_shortest:
        dx, dy = _xy1_[0] - _xy0_[0], _xy1_[1] - _xy0_[1]
        _x_, _y_ = _xy0_[0] + 3.0*dx/4.0, _xy0_[1] + 3.0*dy/4.0
        #svg.append(f'<circle cx="{xT(_x_)}" cy="{yT(_y_)}" r="1.2" fill="#0000ff"/>')
        _x_, _y_ = _xy0_[0] + 1.0*dx/4.0, _xy0_[1] + 1.0*dy/4.0
        #svg.append(f'<circle cx="{xT(_x_)}" cy="{yT(_y_)}" r="1.2" fill="#0000ff"/>')

g_skeleton = nx.Graph()
qt = rt.xyQuadTree((xmin,ymin,xmax,ymax), 30)
qt.add(pts)
for pt in pts:
    nbor_tuples = qt.closest(pt, 6)
    for i in range(1, len(nbor_tuples)):
        nbor_tuple = nbor_tuples[i]
        nbor_pt    = nbor_tuple[1]
        g_skeleton.add_edge(pt, nbor_pt, weight=1.0 / (1.0 + sqrt((pt[0] - nbor_pt[0])**2 + (pt[1] - nbor_pt[1])**2)))
        svg.append(f'<line x1="{xT(pt[0])}" y1="{yT(pt[1])}" x2="{xT(nbor_pt[0])}" y2="{yT(nbor_pt[1])}" stroke-width="0.2" stroke="#0000ff" />')

svg.append('</svg>')
rt.tile([''.join(svg), rt.link(df, [('fm','to')], pos, node_size=0.8, link_size=0.1,w=600, h=600)])

In [ ]:
_pts_   = nx.shortest_path(g_skeleton, pts[1], pts[30])
_trans_ = []
for i in range(len(_pts_)): _trans_.append((xT(_pts_[i][0]), yT(_pts_[i][1])))
_pts_ = _trans_

svg = [f'<svg width="{w}" height="{h}" x="0" y="0">']
for i in range(len(_pts_)-1):
    _xy0_, _xy1_ = _pts_[i], _pts_[i+1]
    svg.append(f'<line x1="{_xy0_[0]}" y1="{_xy0_[1]}" x2="{_xy1_[0]}" y2="{_xy1_[1]}" stroke-width="1.0" stroke="#000000" />')
svg.append(f'<path d="{rt.svgPathCubicBSpline(_pts_, beta=0.9)}" stroke="#ff0000" fill="none" />')
svg.append('</svg>')
rt.tile([''.join(svg)])